In [1]:
import matplotlib.pyplot as plt
import mplfinance
import numpy as np

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

import datetime
import yfinance as yf
import time
import stockstats
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import iplot, init_notebook_mode
from collections import OrderedDict
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
#grouping by frequency 
def FrameChanging(df, x):
    t = df.groupby(pd.Grouper(freq=x)).agg({"open": "first", 
                                             "close": "last", 
                                             "low": "min", 
                                             "high": "max",
                                             "volume": "sum"})
    return t

In [5]:
#adding column dateindex of data type datetime to set as index of data frame
def DatePre(df):
    df['DateIndex'] = pd.to_datetime(df['timestamp'])
    df = df.set_index('DateIndex')
    df = df.drop(['timestamp'], axis=1)
    return df

In [6]:
df = pd.read_csv("../Data/ADANIPOWER__EQ__NSE__NSE__MINUTE.csv")

In [7]:
#converting timestamp to datetime format 
df['DateIndex'] = pd.to_datetime(df['timestamp'])
df = df.set_index('DateIndex')
df = df.drop(['timestamp'], axis=1)

In [8]:
#calling the function for 30 min frequency
df1 = FrameChanging(df, '30Min')

In [9]:
df1 = df1.dropna()

In [10]:
def GetChartPattern(df):
    
    df1 = df.copy()
    
    #moving average calculated over 100 200 and 50 days
    df1['MA100'] = df.rolling(window=100,min_periods=1 ).mean()['close']
    df1['MA200'] = df.rolling(window=200,min_periods=1 ).mean()['close']
    df1['MA50'] = df.rolling(window=50,min_periods=1 ).mean()['close']
    C_DownTrend = True
    C_UpTrend = True

    #marking C_DownTrend and C_UpTrend as true or false according to respective conditions and patterns
    df1['C_DownTrend'] = np.where((  (df1['close'] < df1['MA50']) & (df1['MA50'] < df1['MA200']) ), True, False)
    df1['C_UpTrend'] = np.where(( (df1['close'] > df1['MA50'])& (df1['MA50'] > df1['MA200']) ), True, False)
    C_Len = 14 #ema depth for bodyAvg
    C_ShadowPercent = 5.0 # size of shadows
    C_ShadowEqualsPercent = 100.0
    C_DojiBodyPercent = 5.0
    C_Factor = 2.0 # shows the number of times the shadow dominates the candlestick body
    
    #finding body high and low and body length from closing and opening price of the day
    df1['C_BodyHi'] = df1[['close','open']].max(axis=1)
    df1['C_BodyLo'] = df1[['close','open']].min(axis=1)
    df1['C_Body'] = df1['C_BodyHi'] - df1['C_BodyLo']
    
    df1['C_BodyAvg'] = pd.Series.ewm(df1['C_Body'], span=C_Len).mean()
    
    #setting C_SmallBody as true if body average is > the body length and false if body average is <= to body length
    df1.loc[df1['C_BodyAvg']> df1['C_Body']  , 'C_SmallBody'] = True
    df1.loc[df1['C_BodyAvg']<= df1['C_Body']  , 'C_SmallBody'] = False


    #setting C_LongBody as true if body average is < the body length and false if body average is >= to body length
    df1.loc[df1['C_BodyAvg']< df1['C_Body']  , 'C_LongBody'] = True
    df1.loc[df1['C_BodyAvg']>= df1['C_Body']  , 'C_LongBody'] = False
    
    #calculating upshadow as high - body high and down shadow as body low - low
    df1['C_UpShadow'] = df1['high'] - df1['C_BodyHi']
    df1['C_DnShadow'] = df1['C_BodyLo'] - df1['low']

    #setting C_HasUpShadow as true or false indicating presence of up shadow
    df1['C_HasUpShadow'] = np.where((df1.C_UpShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    #setting C_HasDnShadow as 0 or 1 indicating presence of down shadow                                                
    df1['C_HasDnShadow'] = np.where((df1.C_DnShadow  > (C_ShadowPercent / (100*df1.C_Body))  ),
                                                     True, False)
    
    #white body reprecents traditional green candlestick and black represents red
    df1['C_WhiteBody'] = np.where((df1.open  < df1.close ),True, False)
    df1['C_BlackBody'] = np.where((df1.open  > df1.close ),True, False)
    
    df1['C_Range'] = df1['high']-df1['low']
    df1['C_BodyMiddle']= df1['C_Body'] / (2 + df1['C_BodyLo'])
    
    #findng if upshadow is equal to dowm shadow
    df1['C_ShadowEquals'] = np.where(  ((df1['C_UpShadow'] == df1['C_DnShadow']) | 
                                 (( abs(df1['C_UpShadow'] - df1['C_DnShadow']) / (df1['C_DnShadow'] * 100) ) )  < 
                                 C_ShadowEqualsPercent) & (((abs(df1['C_DnShadow'] - df1['C_UpShadow']) / (df1['C_UpShadow'] * 100))
                                 < C_ShadowEqualsPercent))  , True, False)
    
    #checking if body range is <= 20 percent of body range of candlestick and setting doji body as true or false accordingly
    df1['C_IsDojiBody'] = np.where(( (df1['C_Range'] > 0)  & (df1['C_Body'] <= (df1['C_Range'] * (C_DojiBodyPercent / 100))) ), True, False)
    
    df1['C_Doji'] = df1['C_IsDojiBody'] & df1['C_ShadowEquals']
    
    df1['hl2'] = (df['high'] + df1['low'])/2
    
    #setting the further columns as true or false according to the respective conditions

    df1['C_GravestoneDojiOne'] = np.where( (  df1['C_IsDojiBody'] & (df1['C_DnShadow'] <= df1['C_Body'] )  )   , True, False)
    df1['C_DragonflyDoji'] = np.where( (  df1['C_IsDojiBody'] & (df1['C_UpShadow'] <= df1['C_Body'] )  )   , True, False)
    
    C_SpinningTopBlackPercent = 34.0
    C_SpinningTopWhitePercent = 34.0
    
    
    
    
    df1['C_IsSpinningTop'] = np.where( (( df1['C_DnShadow'] >= (df1['C_Range'] / (100 * C_SpinningTopBlackPercent)) ) &
                                      (df1['C_UpShadow'] >= (df1['C_Range'] / (100 * C_SpinningTopBlackPercent)) ) &
                                     (~df1['C_IsDojiBody'] ) )  , True, False )
    
    df1['C_3Dojis'] = np.where( (df1['C_Doji'].shift(2) & df1['C_Doji'].shift(1) & df1['C_Doji'] )  , True, False )
    df1['C_BodyGapUp'] = np.where( (df1['C_BodyHi'].shift(1) < df1['C_BodyLo'] )  , True, False )
    df1['C_BodyGapDn'] = np.where( (df1['C_BodyLo'].shift(1) > df1['C_BodyHi'] )  , True, False )
    
    df1['C_IsSpinningTopWhite'] = np.where( (( df1['C_DnShadow'] >= (df1['C_Range'] / (100 * C_SpinningTopWhitePercent)) ) &
                                      (df1['C_UpShadow'] >= (df1['C_Range'] / (100 * C_SpinningTopWhitePercent)) ) &
                                     (~df1['C_IsDojiBody'] ) )  , True, False )
   
    
    
    
    df1['C_AbandonedBabyBearish'] = np.where( ( ( df1['C_UpTrend'].shift(2)  & df1['C_WhiteBody'].shift(2) & 
                                             df1['C_IsDojiBody'].shift(1) &  (df1['high'].shift(2) <df1['low'].shift(1)) &
                                             df1['C_BlackBody'] & (df1['low'].shift(1) > df1['high'] )   ) )    , "1", "0" )
    
    df1['C_AbandonedBabyBullish'] = np.where( ( ( df1['C_DownTrend'].shift(2)  & df1['C_BlackBody'].shift(2) & 
                                             df1['C_IsDojiBody'].shift(1) &  (df1['low'].shift(2) > df1['high'].shift(1)) &
                                             df1['C_WhiteBody'] & (df1['high'].shift(1) < df1['low'] )   ) )    , "1", "0" )
    
    
    
    df1['C_HangingManBearish'] = np.where( (( df1['C_UpTrend']  & df1['C_SmallBody'] & 
                                             df1['C_Body'] > 0 &  (df1['hl2'] < df1['C_BodyLo']) &
                                             (df1['C_DnShadow'] >= (C_Factor * df1['C_Body'])) & (~ df1['C_HasUpShadow'])  ) ), "1", "0" )
    
    
    
    
    df1['C_DojiStarBearish'] = np.where( (( df1['C_UpTrend']  & df1['C_WhiteBody'].shift(1) & 
                                             df1['C_LongBody'].shift(1) &  df1['C_IsDojiBody'] &
                                             (df1['C_BodyLo'] >  df1['C_BodyHi'].shift(1) )  ) ), "1", "0" )
    
    
    
    df1['C_DojiStarBullish'] = np.where( (( df1['C_DownTrend']  & df1['C_BlackBody'].shift(1) & 
                                             df1['C_LongBody'].shift(1) &  df1['C_IsDojiBody'] &
                                             (df1['C_BodyLo'].shift(1) >  df1['C_BodyHi'])  ) ), "1", "0" )
    
    
    
    df1['C_DarkCloudCoverBearish'] = np.where( (  ( df1['C_UpTrend'].shift(1)  & df1['C_WhiteBody'].shift(1) & 
                                             df1['C_LongBody'].shift(1))     &  ( df1['C_BlackBody'] &
                                             ( df1['open'] >=  df1['high'].shift(1)) & (df1['close']< df1['C_BodyMiddle'].shift(1)) & 
                                                        (df1['close'] > df1['open'].shift(1)) ) ), "1", "0" )
    
    

    
    
    df1['C_DojiRes'] = np.where( ( df1['C_Doji'] & (~ df1['C_DragonflyDoji']) & (~ df1['C_GravestoneDojiOne']) ), "1", "0" )
    


    
    df1['C_EngulfingBearish'] = np.where( ( df1['C_UpTrend']  &  df1['C_BlackBody'] &
                                          df1['C_LongBody'] &  df1['C_WhiteBody'].shift(1) &
                                          df1['C_SmallBody'].shift(1) &
                                            (df1['close'] <= df1['open'].shift(1))  &
                                           (df1['open'] >= df1['close'].shift(1)) &
                                          ( (df1['close'] <= df1['open'].shift(1)) | (df1['open'] > df1['close'].shift(1)) ))   , "1", "0")
    
    
    df1['C_EngulfingBullish'] = np.where( ( df1['C_DownTrend']  &  df1['C_WhiteBody'] &
                                           df1['C_LongBody'] &  df1['C_BlackBody'].shift(1) &
                                          df1['C_SmallBody'].shift(1) &
                                            (df1['close'] >= df1['open'].shift(1))  &
                                           (df1['open'] <= df1['close'].shift(1)) &
                                          ( (df1['close'] > df1['open'].shift(1)) | (df1['open'] <df1['close'].shift(1)) ))   , "1", "0")
    
    
    
    df1['C_EveningDojiStarBearish'] = np.where(( df1['C_LongBody'].shift(2) & df1['C_IsDojiBody'].shift(1) &
                                            df1['C_LongBody'] & df1['C_UpTrend'] &  df1['C_WhiteBody'] &
                                             (df1['C_BodyLo'].shift(1) > df1['C_BodyHi'].shift(2)) &
                                              df1['C_BlackBody'] &  (df1['C_BodyLo'] <= df1['C_BodyMiddle'].shift(2)) &
                                              (df1['C_BodyLo'] > df1['C_BodyLo'].shift(2)) & 
                                               (df1['C_BodyLo'].shift(2) > df1['C_BodyHi'])), "1", "0" )
    
   


    df1['C_EveningStarBearish'] = np.where((df1['C_LongBody'].shift(2) &
                                            df1['C_SmallBody'].shift(1) & 
                                            df1['C_LongBody'] &
                                           df1['C_UpTrend'] & df1['C_WhiteBody'].shift(2) &
                                           (df1['C_BodyLo'].shift(1) > df1['C_BodyHi'].shift(2)) &
                                           df1['C_BlackBody'] & 
                                            (df1['C_BodyLo'] <= df1['C_BodyMiddle'].shift(2)) &
                                           (df1['C_BodyLo'] > df1['C_BodyLo'].shift(2)) &
                                           (df1['C_BodyLo'].shift(1) > df1['C_BodyHi'])), "1", "0" )
    

    df1['C_FallingWindowBearish'] = np.where((df1['C_DownTrend'].shift(1) &
                                               ( (df1['C_Range'] != 0) & (df1['C_Range'].shift(1) != 0) ) &
                                               (df1['high'] < df1['low'].shift(1)) ), "1", "0" )
    
  
    
    
    df1['C_GravestoneDojiBearish'] = np.where( (  df1['C_IsDojiBody'] & (df1['C_DnShadow'] <= df1['C_Body'] )  )   , "1", "0")
  
    
    
    
    df1['C_HammerBullish'] = np.where( (  df1['C_SmallBody'] & (df1['C_Body'] > 0) &
                                       (df1['C_BodyLo'] > df1['hl2'] ) &
                                      (df1['C_DnShadow'] >= C_Factor * df1['C_Body']) &
                                       (~ df1['C_HasUpShadow']) &
                                       df1['C_DownTrend'] )   , "1", "0")
    
    df1['C_HaramiBearish'] = np.where( ( df1['C_LongBody'].shift(1) & df1['C_WhiteBody'].shift(1) &
                                      df1['C_UpTrend'].shift(1) & df1['C_BlackBody'] & df1['C_SmallBody'] &
                                       (df1['high'] <= df1['C_BodyHi'].shift(1)) &
                                       (df1['low'] >= df1['C_BodyLo'].shift(1)) )   , "1", "0")
  
    
    
    
    df1['C_HaramiBullish'] = np.where( ( df1['C_LongBody'].shift(1) & df1['C_BlackBody'].shift(1) &
                                      df1['C_DownTrend'].shift(1) & df1['C_WhiteBody'] & df1['C_SmallBody'] &
                                       (df1['high'] <= df1['C_BodyHi'].shift(1)) &
                                       (df1['low'] >= df1['C_BodyLo'].shift(1)) )   , "1", "0")
  
    
    
    df1['C_HaramiCrossBearish'] = np.where( ( df1['C_LongBody'].shift(1) & df1['C_WhiteBody'].shift(1) &
                                      df1['C_UpTrend'].shift(1) & df1['C_IsDojiBody'] &
                                       (df1['high'] <= df1['C_BodyHi'].shift(1)) &
                                       (df1['low'] >= df1['C_BodyLo'].shift(1)) )   , "1", "0")
  

    
    df1['C_HaramiCrossBullish'] = np.where( ( df1['C_LongBody'].shift(1) & df1['C_BlackBody'].shift(1) &
                                      df1['C_DownTrend'].shift(1) & df1['C_IsDojiBody']  &
                                       (df1['high'] <= df1['C_BodyHi'].shift(1)) &
                                       (df1['low'] >= df1['C_BodyLo'].shift(1)) )   , "1", "0")
  


    df1['C_InvertedHammerBullish'] = np.where( ( df1['C_SmallBody'] & (df1['C_Body'] > 0) &
                                              (df1['C_BodyHi'] < df1['hl2']) &
                                              (df1['C_UpShadow'] >= C_Factor * df1['C_Body']) &
                                               (~ df1['C_HasDnShadow']))   , "1", "0")
  
    
    
    C_LongLowerShadowPercent = 75.0

    
    df1['C_LongLowerShadowBullish'] = np.where( ( df1['C_DnShadow'] > (df1['C_Range'] / (100*C_LongLowerShadowPercent)) )   , "1", "0")
  
    
    C_LongLowerShadowPercent = 75.0

    df1['C_LongUpperShadowBearish'] = np.where( ( df1['C_UpShadow'] > (df1['C_Range'] / (100*C_LongLowerShadowPercent)) )   , "1", "0")
  
    
    
    C_MarubozuShadowPercentBearish = 5.0

    df1['C_MarubozuBlackBearish'] = np.where( ( df1['C_BlackBody'] & df1['C_LongBody'] &
                                           (df1['C_UpShadow'] <= C_MarubozuShadowPercentBearish/(100*df1['C_Body']) ) &
                                            (df1['C_DnShadow'] <= C_MarubozuShadowPercentBearish/(100*df1['C_Body'])) &
                                            df1['C_BlackBody']  )   , "1", "0")
  
    
    
    C_MarubozuShadowPercentWhite = 5.0
    df1['MarubozuWhiteBullish'] = np.where( ( df1['C_WhiteBody'] & df1['C_LongBody'] &
                                           (df1['C_UpShadow'] <= C_MarubozuShadowPercentWhite/(100*df1['C_Body']) ) &
                                            (df1['C_DnShadow'] <= C_MarubozuShadowPercentWhite/(100*df1['C_Body'])) &
                                            df1['C_WhiteBody']  )   , "1", "0")
  
   
    df1['C_MorningDojiStarBullish'] = np.where(( df1['C_LongBody'].shift(2) & df1['C_IsDojiBody'].shift(1) &
                                            df1['C_LongBody'] & df1['C_DownTrend'] &  df1['C_BlackBody'].shift(2) &
                                             (df1['C_BodyHi'].shift(1) < df1['C_BodyLo'].shift(2)) &
                                              df1['C_WhiteBody'] &  (df1['C_BodyHi'] >= df1['C_BodyMiddle'].shift(2)) &
                                              (df1['C_BodyHi'] < df1['C_BodyHi'].shift(2)) & 
                                               (df1['C_BodyHi'].shift(1) < df1['C_BodyLo'])), "1", "0" )
    
    
    
    df1['C_MorningStarBullish'] = np.where(( df1['C_LongBody'].shift(2) & df1['C_SmallBody'].shift(1) &
                                            df1['C_LongBody'] & df1['C_DownTrend'] &  df1['C_BlackBody'].shift(2) &
                                             (df1['C_BodyHi'].shift(1) < df1['C_BodyLo'].shift(2)) &
                                              df1['C_WhiteBody'] &  (df1['C_BodyHi'] >= df1['C_BodyMiddle'].shift(2)) &
                                              (df1['C_BodyHi'] < df1['C_BodyHi'].shift(2)) & 
                                               (df1['C_BodyHi'].shift(1) < df1['C_BodyLo'])), "1", "0" )
    
    
    
    df1['C_OnNeckBearish'] = np.where((df1['C_DownTrend'] & df1['C_BlackBody'].shift(1) &
                                     df1['C_LongBody'].shift(1) & df1['C_WhiteBody'] & 
                                      (df1['open'] < df1['close'].shift(1)) &
                                     df1['C_SmallBody'] & ( df1['C_Range'] != 0 ) &
                                     (abs(df1['close'] - df1['low'].shift(1)) <= df1['C_BodyAvg']*0.05) ), "1", "0" )
    
    
    df1['C_PiercingBullish'] = np.where( ( (df1['C_DownTrend'].shift(1) & df1['C_BlackBody'].shift(1) & df1['C_LongBody'].shift(1)) &
                                        ( (df1['C_WhiteBody'] & df1['open'] <= df1['low'].shift(1)) & 
                                         (df1['close'] > df1['C_BodyMiddle'].shift(1)) & 
                                         (df1['close'] < df1['open'].shift(1)) )  )  , "1", "0" )
    
    
    df1['C_RisingWindowBullish'] = np.where( ( ( (df1['C_Range'] !=0) & (df1['C_Range'].shift(1)!=0 ) ) &
                                             df1['C_UpTrend'].shift(1) &
                                            df1['low'] > df1['high'].shift(1) )  , "1", "0" )
    
    
    
    df1['C_ShootingStarBearish'] = np.where( (df1['C_SmallBody'] & (df1['C_Body'] > 0 ) &
                                              (df1['C_BodyHi'] < df1['hl2']) &
                                             (df1['C_UpShadow'] >= C_Factor * df1['C_Body']) &
                                            (~ df1['C_HasDnShadow']) )  , "1", "0" )
    
    

   
    df1['C_SpinningTopBlack'] = np.where((df1['C_IsSpinningTop'] & df1['C_BlackBody']) , "1", "0")
    
    
    
    
 
    
   
    df1['C_SpinningTopWhite'] = np.where((df1['C_IsSpinningTopWhite'] & df1['C_WhiteBody']) , "1", "0")
    
    
    
    
    
   

    df1['C_TriStarBearish'] = np.where(( df1['C_3Dojis'] &  df1['C_UpTrend'].shift(2) &
                                       df1['C_BodyGapUp'].shift(1) & df1['C_BodyGapDn'] ) , "1", "0")
    
    
    df1['C_TriStarBullish'] = np.where(( df1['C_3Dojis'] &  df1['C_DownTrend'].shift(2) &
                                       df1['C_BodyGapDn'].shift(1) & df1['C_BodyGapUp'] ) , "1", "0")
    
    
    
    df1['C_TweezerBottomBullish'] = np.where(( df1['C_UpTrend'].shift(1) &
                                           ( (~df1['C_IsDojiBody']) | (df1['C_HasUpShadow'] & df1['C_HasDnShadow']) )  &
                                            (abs(df1['low']-df1['low'].shift(1)) <= (df1['C_BodyAvg']*0.05) ) &
                                            df1['C_BlackBody'].shift(1) & df1['C_WhiteBody'] &  df1['C_LongBody'].shift(1) ) , "1", "0")
    
    
    df1['C_TweezerTopBearish'] = np.where(( df1['C_UpTrend'].shift(1) &
                                           ( (~df1['C_IsDojiBody']) | (df1['C_HasUpShadow'] & df1['C_HasDnShadow']) )  &
                                            (abs(df1['high']-df1['high'].shift(1)) <= (df1['C_BodyAvg']*0.05) ) &
                                            df1['C_WhiteBody'].shift(1) & df1['C_BlackBody'] &  df1['C_LongBody'].shift(1) ) , "1", "0")
    
    
    
    
    return df1

In [11]:
#for all the mentioned columns the values are normallised 
def normalize(df):
    result = df.copy()
    for feature_name in ['open','close','low','high','volume','MA100','MA200','MA50', 'C_BodyHi', 'C_BodyLo', 'hl2']:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [27]:
def Preprocess(df):
    
    df['DateIndex'] = pd.to_datetime(df['timestamp'])#convert timestamp to date time formant and save it to date index
    df = df.set_index('DateIndex')#set dateindex as index of data frame
    df = df.drop(['timestamp'], axis=1)
    df1 = FrameChanging(df, '30Min')#grouping by frequeny of 30 minutes
    df1 = df1.dropna()
    #calculating lag,return,P_result1,P_result0 
    df1['L1'] = df1['close'].shift(1)
    df1['Return'] = (df1['close']-df1['L1'])
    
    df1.loc[df1['Return']>=0, 'Result_1'] = '1'
    
    df1.loc[df1['Return']<0, 'Result_0'] = '0'
    
    df1['P_Result_1'] = 0
    df1['P_Result_0'] = 0
    df1['P_Result_1'] = df1['Result_1'].shift(-1)
    df1['P_Result_0'] = df1['Result_0'].shift(-1)
    
    df1['P_Result']  = df1['P_Result'].astype('category')
    
    dfR = GetChartPattern(df1)#generating all the candlestick pattern columns in the function
    
    dfRS = normalize(dfR)#normalise the data frame
    #replace true with 1 and false with 0
    dfRS = dfRS.replace(True,1)
    dfRS = dfRS.replace(False,0)
    
    return dfRS
    
    

In [28]:
import glob
l =(glob.glob("../Data/*.csv"))
print(l[1])

../Data/TORNTPHARM__EQ__NSE__NSE__MINUTE.csv


In [29]:
Num = len(l)
Num

160

In [30]:
l

['../Data/INFY__EQ__NSE__NSE__MINUTE.csv',
 '../Data/TORNTPHARM__EQ__NSE__NSE__MINUTE.csv',
 '../Data/MGL__EQ__NSE__NSE__MINUTE.csv',
 '../Data/CHOLAFIN__EQ__NSE__NSE__MINUTE.csv',
 '../Data/DRREDDY__EQ__NSE__NSE__MINUTE.csv',
 '../Data/INDIANB__EQ__NSE__NSE__MINUTE.csv',
 '../Data/GMRINFRA__EQ__NSE__NSE__MINUTE.csv',
 '../Data/AUBANK__EQ__NSE__NSE__MINUTE.csv',
 '../Data/MUTHOOTFIN__EQ__NSE__NSE__MINUTE.csv',
 '../Data/RAMCOCEM__EQ__NSE__NSE__MINUTE.csv',
 '../Data/APOLLOHOSP__EQ__NSE__NSE__MINUTE.csv',
 '../Data/IDBI__EQ__NSE__NSE__MINUTE.csv',
 '../Data/NIFTY_500__EQ__INDICES__NSE__MINUTE.csv',
 '../Data/MARUTI__EQ__NSE__NSE__MINUTE.csv',
 '../Data/SAIL__EQ__NSE__NSE__MINUTE.csv',
 '../Data/VBL__EQ__NSE__NSE__MINUTE.csv',
 '../Data/ICICIBANK__EQ__NSE__NSE__MINUTE.csv',
 '../Data/HCLTECH__EQ__NSE__NSE__MINUTE.csv',
 '../Data/BAJFINANCE__EQ__NSE__NSE__MINUTE.csv',
 '../Data/BHARATFORG__EQ__NSE__NSE__MINUTE.csv',
 '../Data/GRAPHITE__EQ__NSE__NSE__MINUTE.csv',
 '../Data/HDFC__EQ__NSE__N

In [31]:
l[1]

'../Data/TORNTPHARM__EQ__NSE__NSE__MINUTE.csv'

In [32]:
df = pd.read_csv(l[1])

In [33]:
df = pd.read_csv(l[0])
df1 = Preprocess(df)

KeyError: 'P_Result'

In [ ]:
#for all the csvs listed,the data frams is preprocessed and concatinated with df1
for i in range(0,int(Num/10) ):
    print(i)
    try:
        df = pd.read_csv(l[i])
        dfP = Preprocess(df)
        df1 = pd.concat([df1, dfP], axis=0)
    except:
        print(i , " failed")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [ ]:
#displaying how many data points satisfy the respective conditions for each pattern
for i in range(28, df1.shape[1]):
    print(  (list(df1))[i])
    print(df1[(list(df1))[i]].value_counts())

C_GravestoneDojiOne
0.0    182866
1.0       562
Name: C_GravestoneDojiOne, dtype: int64
C_DragonflyDoji
0.0    182843
1.0       585
Name: C_DragonflyDoji, dtype: int64
C_IsSpinningTop
1.0    138440
0.0     44988
Name: C_IsSpinningTop, dtype: int64
C_3Dojis
0.0    183268
1.0       160
Name: C_3Dojis, dtype: int64
C_BodyGapUp
0.0    172296
1.0     11132
Name: C_BodyGapUp, dtype: int64
C_BodyGapDn
0.0    174497
1.0      8931
Name: C_BodyGapDn, dtype: int64
C_IsSpinningTopWhite
1.0    138440
0.0     44988
Name: C_IsSpinningTopWhite, dtype: int64
C_AbandonedBabyBearish
0    183428
Name: C_AbandonedBabyBearish, dtype: int64
C_AbandonedBabyBullish
0    183428
Name: C_AbandonedBabyBullish, dtype: int64
C_HangingManBearish
0    149979
1     33449
Name: C_HangingManBearish, dtype: int64
C_DojiStarBearish
0    183295
1       133
Name: C_DojiStarBearish, dtype: int64
C_DojiStarBullish
0    183330
1        98
Name: C_DojiStarBullish, dtype: int64
C_DarkCloudCoverBearish
0    183428
Name: C_DarkClou

In [ ]:
df1.shape

(183428, 70)

In [ ]:
Dataset = df1.to_numpy()

In [ ]:
list(df1)

['open',
 'close',
 'low',
 'high',
 'volume',
 'MA100',
 'MA200',
 'MA50',
 'C_DownTrend',
 'C_UpTrend',
 'C_BodyHi',
 'C_BodyLo',
 'C_Body',
 'C_BodyAvg',
 'C_SmallBody',
 'C_LongBody',
 'C_UpShadow',
 'C_DnShadow',
 'C_HasUpShadow',
 'C_HasDnShadow',
 'C_WhiteBody',
 'C_BlackBody',
 'C_Range',
 'C_BodyMiddle',
 'C_ShadowEquals',
 'C_IsDojiBody',
 'C_Doji',
 'hl2',
 'C_GravestoneDojiOne',
 'C_DragonflyDoji',
 'C_IsSpinningTop',
 'C_3Dojis',
 'C_BodyGapUp',
 'C_BodyGapDn',
 'C_IsSpinningTopWhite',
 'C_AbandonedBabyBearish',
 'C_AbandonedBabyBullish',
 'C_HangingManBearish',
 'C_DojiStarBearish',
 'C_DojiStarBullish',
 'C_DarkCloudCoverBearish',
 'C_DojiRes',
 'C_EngulfingBearish',
 'C_EngulfingBullish',
 'C_EveningDojiStarBearish',
 'C_EveningStarBearish',
 'C_FallingWindowBearish',
 'C_GravestoneDojiBearish',
 'C_HammerBullish',
 'C_HaramiBearish',
 'C_HaramiBullish',
 'C_HaramiCrossBearish',
 'C_HaramiCrossBullish',
 'C_InvertedHammerBullish',
 'C_LongLowerShadowBullish',
 'C_LongUp

In [ ]:
X = Dataset[:,:28]
Y = Dataset[:, 28:38]
seq_len = 6
x = []
y = []

for i in range(seq_len, len(X)):
    x.append(X[i-seq_len:i])
    
    yt = Y[i-seq_len:i]
    yr = []
    res = np.sum(yt, axis = 0)
    for i in range(0, len(res)):
        if(res[i] ==0):
            yr.append(0)
        else:
            yr.append(1)
    y.append(yr)

NameError: name 'Dataset' is not defined

In [ ]:
x1 = np.asarray(x).astype('float32')
y1 = np.asarray(y).astype('float32')
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x1, y1, train_size=0.8, random_state=42)
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_test.shape[1]


In [ ]:
from tensorflow import keras
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import LSTM
import keras.models
from keras.layers import Activation, Dense
from keras.models import Sequential

In [ ]:
epochs = 20
batch_size =64


In [ ]:
model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# fit network
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
          verbose=0, validation_data=(X_test, y_test))

In [ ]:
history.history

{'loss': [10.379304885864258,
  10.295230865478516,
  10.278900146484375,
  10.268689155578613,
  10.262412071228027,
  10.256577491760254,
  10.251914978027344,
  10.248526573181152,
  10.244906425476074,
  10.24172306060791,
  10.239846229553223,
  10.237640380859375,
  10.234970092773438,
  10.232460021972656,
  10.230527877807617,
  10.227298736572266,
  10.22553825378418,
  10.223241806030273,
  10.220040321350098,
  10.21804428100586],
 'accuracy': [0.012934706173837185,
  0.002221661852672696,
  0.003250713925808668,
  0.005915345158427954,
  0.0076122586615383625,
  0.006705875042825937,
  0.01080845296382904,
  0.020213034003973007,
  0.013861534185707569,
  0.02559681609272957,
  0.021664610132575035,
  0.026741720736026764,
  0.062356460839509964,
  0.07349884510040283,
  0.05303365737199783,
  0.08498197793960571,
  0.08680836856365204,
  0.10467707365751266,
  0.09872765839099884,
  0.08708097040653229],
 'val_loss': [10.30299186706543,
  10.286443710327148,
  10.281242370

In [ ]:
# evaluate model
_, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
Y_pred = model.predict(X_test)

In [ ]:
accuracy

0.07965108007192612

In [ ]:
Y_pred[0]

array([5.9371132e-08, 1.5628187e-06, 9.9994993e-01, 2.1428307e-06,
       5.5973172e-02, 4.7312689e-01, 9.9992383e-01, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00], dtype=float32)

In [ ]:
y_test[0]

array([0., 0., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)

In [ ]:
Y_pred[1]

array([5.68263260e-07, 4.30582531e-07, 9.99961913e-01, 2.53198732e-06,
       3.39698404e-01, 1.11490756e-01, 9.99935985e-01, 9.99999225e-01,
       9.99997914e-01, 9.99999285e-01], dtype=float32)

In [ ]:
y_test[1]

array([0., 0., 1., 0., 0., 0., 1., 1., 1., 1.], dtype=float32)

In [ ]:
list(df1)

['open',
 'close',
 'low',
 'high',
 'volume',
 'MA100',
 'MA200',
 'MA50',
 'C_DownTrend',
 'C_UpTrend',
 'C_BodyHi',
 'C_BodyLo',
 'C_Body',
 'C_BodyAvg',
 'C_SmallBody',
 'C_LongBody',
 'C_UpShadow',
 'C_DnShadow',
 'C_HasUpShadow',
 'C_HasDnShadow',
 'C_WhiteBody',
 'C_BlackBody',
 'C_Range',
 'C_BodyMiddle',
 'C_ShadowEquals',
 'C_IsDojiBody',
 'C_Doji',
 'hl2',
 'C_GravestoneDojiOne',
 'C_DragonflyDoji',
 'C_IsSpinningTop',
 'C_3Dojis',
 'C_BodyGapUp',
 'C_BodyGapDn',
 'C_IsSpinningTopWhite',
 'C_AbandonedBabyBearish',
 'C_AbandonedBabyBullish',
 'C_HangingManBearish',
 'C_DojiStarBearish',
 'C_DojiStarBullish',
 'C_DarkCloudCoverBearish',
 'C_DojiRes',
 'C_EngulfingBearish',
 'C_EngulfingBullish',
 'C_EveningDojiStarBearish',
 'C_EveningStarBearish',
 'C_FallingWindowBearish',
 'C_GravestoneDojiBearish',
 'C_HammerBullish',
 'C_HaramiBearish',
 'C_HaramiBullish',
 'C_HaramiCrossBearish',
 'C_HaramiCrossBullish',
 'C_InvertedHammerBullish',
 'C_LongLowerShadowBullish',
 'C_LongUp

In [ ]:
from plotly.offline import plot
import plotly.graph_objs as go

In [ ]:
X_test.shape

(36685, 6, 28)

In [ ]:
# X = Dataset[:,:28]
# Y = Dataset[:, 28:38]
# seq_len = 6

o = X_test[0, : , 0]

In [ ]:
o

array([0.45014033, 0.44836295, 0.44817588, 0.4484565 , 0.4462114 ,
       0.4393826 ], dtype=float32)

In [ ]:
o = X_test[0, : , 0]
h = X_test[0, : , 3]
l = X_test[0, : , 2]
c = X_test[0, : , 1]

In [ ]:
trace = go.Candlestick(
            open=o,
            high=h,
            low=l,
            close=c)
data = [trace]

plot(data)

'temp-plot.html'

In [ ]:
#Rough Work

In [ ]:
dfR = GetChartPattern(df1)
dfR.shape

(183428, 70)

In [ ]:
dfR.head(100)

,open,close,low,high,volume,MA100,MA200,MA50,C_DownTrend,C_UpTrend,C_BodyHi,C_BodyLo,C_Body,C_BodyAvg,C_SmallBody,C_LongBody,C_UpShadow,C_DnShadow,C_HasUpShadow,C_HasDnShadow,C_WhiteBody,C_BlackBody,C_Range,C_BodyMiddle,C_ShadowEquals,C_IsDojiBody,C_Doji,hl2,C_GravestoneDojiOne,C_DragonflyDoji,C_IsSpinningTop,C_3Dojis,C_BodyGapUp,C_BodyGapDn,C_IsSpinningTopWhite,C_AbandonedBabyBearish,C_AbandonedBabyBullish,C_HangingManBearish,C_DojiStarBearish,C_DojiStarBullish,C_DarkCloudCoverBearish,C_DojiRes,C_EngulfingBearish,C_EngulfingBullish,C_EveningDojiStarBearish,C_EveningStarBearish,C_FallingWindowBearish,C_GravestoneDojiBearish,C_HammerBullish,C_HaramiBearish,C_HaramiBullish,C_HaramiCrossBearish,C_HaramiCrossBullish,C_InvertedHammerBullish,C_LongLowerShadowBullish,C_LongUpperShadowBearish,C_MarubozuBlackBearish,MarubozuWhiteBullish,C_MorningDojiStarBullish,C_MorningStarBullish,C_OnNeckBearish,C_PiercingBullish,C_RisingWindowBullish,C_ShootingStarBearish,C_SpinningTopBlack,C_SpinningTopWhite,C_TriStarBearish,C_TriStarBullish,C_TweezerBottomBullish,C_TweezerTopBearish
DateIndex,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-09-01 09:30:00+05:30,0.976948,1.000000,1.000000,0.976852,0.073068,1.000000,1.000000,1.000000,False,False,1.000000,0.976948,0.023052,0.023052,False,False,-0.023148,-0.023052,False,False,True,False,-0.023148,0.007744,True,False,False,0.988426,False,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-09-01 10:00:00+05:30,1.000000,0.940898,0.941218,1.000000,0.107042,0.970449,0.970449,0.970449,False,False,1.000000,0.940898,0.059102,0.042364,False,True,0.000000,-0.000320,False,False,False,True,0.058782,0.020096,False,False,False,0.970609,False,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-09-01 10:30:00+05:30,0.919318,0.940898,0.941218,0.918981,0.005012,0.960599,0.960599,0.960599,False,False,0.940898,0.919318,0.021581,0.034425,True,False,-0.021917,-0.021900,False,False,True,False,-0.022236,0.007392,True,False,False,0.930100,False,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-09-01 11:00:00+05:30,0.919318,0.940898,0.941218,0.918981,0.003812,0.955674,0.955674,0.955674,False,False,0.940898,0.919318,0.021581,0.030496,True,False,-0.021917,-0.021900,False,False,True,False,-0.022236,0.007392,True,False,False,0.930100,False,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-09-01 11:30:00+05:30,0.919318,0.940898,0.941218,0.918981,0.001380,0.952719,0.952719,0.952719,False,False,0.940898,0.919318,0.021581,0.028170,True,False,-0.021917,-0.021900,False,False,True,False,-0.022236,0.007392,True,False,False,0.930100,False,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-12 11:30:00+05:30,0.718073,0.718676,0.715495,0.716898,0.041917,0.817132,0.817132,0.770619,True,False,0.718676,0.718073,0.000603,0.014443,True,False,-0.001778,0.002578,False,False,True,False,0.001403,0.000222,True,False,False,0.716197,False,False,False,False,False,True,False,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-09-12 12:00:00+05:30,0.702628,0.723404,0.715495,0.706481,0.015466,0.816166,0.816166,0.769466,True,False,0.723404,0.702628,0.020776,0.015288,False,True,-0.016923,-0.012867,False,False,True,False,-0.009013,0.007687,True,False,False,0.710988,False,False,False,False,False,False,False,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2017-09-12 12:30:00+05:30,0.707238,0.717494,0.710792,0.707176,0

In [ ]:
from sklearn import preprocessing

In [ ]:
dfRS = normalize(dfR)

In [ ]:
dfRS = dfRS.replace(True,1)

In [ ]:
dfRS = dfRS.replace(False,0)

In [ ]:
dfRS.head(100)

,open,close,low,high,volume,MA100,MA200,MA50,C_DownTrend,C_UpTrend,C_BodyHi,C_BodyLo,C_Body,C_BodyAvg,C_SmallBody,C_LongBody,C_UpShadow,C_DnShadow,C_HasUpShadow,C_HasDnShadow,C_WhiteBody,C_BlackBody,C_Range,C_BodyMiddle,C_ShadowEquals,C_IsDojiBody,C_Doji,hl2,C_GravestoneDojiOne,C_DragonflyDoji,C_IsSpinningTop,C_3Dojis,C_BodyGapUp,C_BodyGapDn,C_IsSpinningTopWhite,C_AbandonedBabyBearish,C_AbandonedBabyBullish,C_HangingManBearish,C_DojiStarBearish,C_DojiStarBullish,C_DarkCloudCoverBearish,C_DojiRes,C_EngulfingBearish,C_EngulfingBullish,C_EveningDojiStarBearish,C_EveningStarBearish,C_FallingWindowBearish,C_GravestoneDojiBearish,C_HammerBullish,C_HaramiBearish,C_HaramiBullish,C_HaramiCrossBearish,C_HaramiCrossBullish,C_InvertedHammerBullish,C_LongLowerShadowBullish,C_LongUpperShadowBearish,C_MarubozuBlackBearish,MarubozuWhiteBullish,C_MorningDojiStarBullish,C_MorningStarBullish,C_OnNeckBearish,C_PiercingBullish,C_RisingWindowBullish,C_ShootingStarBearish,C_SpinningTopBlack,C_SpinningTopWhite,C_TriStarBearish,C_TriStarBullish,C_TweezerBottomBullish,C_TweezerTopBearish
DateIndex,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-09-01 09:30:00+05:30,0.976948,1.000000,1.000000,0.976852,0.073068,1.000000,1.000000,1.000000,0.0,0.0,1.000000,0.978129,0.023052,0.023052,0,0,-0.023148,-0.023052,0.0,0.0,1.0,0.0,-0.023148,0.007744,1.0,0.0,0.0,0.988426,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-09-01 10:00:00+05:30,1.000000,0.940898,0.941218,1.000000,0.107042,0.970186,0.970073,0.970260,0.0,0.0,1.000000,0.942035,0.059102,0.042364,0,1,0.000000,-0.000320,0.0,0.0,0.0,1.0,0.058782,0.020096,0.0,0.0,0.0,0.970609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-09-01 10:30:00+05:30,0.919318,0.940898,0.941218,0.918981,0.005012,0.960248,0.960098,0.960346,0.0,0.0,0.940868,0.920429,0.021581,0.034425,1,0,-0.021917,-0.021900,0.0,0.0,1.0,0.0,-0.022236,0.007392,1.0,0.0,0.0,0.930100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-09-01 11:00:00+05:30,0.919318,0.940898,0.941218,0.918981,0.003812,0.955279,0.955110,0.955390,0.0,0.0,0.940868,0.920429,0.021581,0.030496,1,0,-0.021917,-0.021900,0.0,0.0,1.0,0.0,-0.022236,0.007392,1.0,0.0,0.0,0.930100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-09-01 11:30:00+05:30,0.919318,0.940898,0.941218,0.918981,0.001380,0.952298,0.952117,0.952416,0.0,0.0,0.940868,0.920429,0.021581,0.028170,1,0,-0.021917,-0.021900,0.0,0.0,1.0,0.0,-0.022236,0.007392,1.0,0.0,0.0,0.930100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-09-12 11:30:00+05:30,0.718073,0.718676,0.715495,0.716898,0.041917,0.815504,0.814806,0.769149,1.0,0.0,0.718532,0.718941,0.000603,0.014443,1,0,-0.001778,0.002578,0.0,0.0,1.0,0.0,0.001403,0.000222,1.0,0.0,0.0,0.716197,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2017-09-12 12:00:00+05:30,0.702628,0.723404,0.715495,0.706481,0.015466,0.814529,0.813827,0.767988,1.0,0.0,0.723263,0.703477,0.020776,0.015288,0,1,-0.016923,-0.012867,0.0,0.0,1.0,0.0,-0.009013,0.007687,1.0,0.0,0.0,0.710988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2017-09-12 12:30:00+05:30,0.707238,0.717494,0.710792,0.707176,0.022308,0.813513,0.812808,0.766708,1.0,0.0,0.717350,0.708093,0.010256,0.014617,1,0,-0.010318,-0.003554,0.0,0.0,1.0,0.0,-0.003616,0.003788,1.0,0.0,0.0,0.708984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Zero Ahead classification (42)

data = dfRS.to_numpy()



In [ ]:
data.shape

(183428, 70)

In [ ]:
X = data[:,:28]

In [ ]:
Y = data[:, 28:]

In [ ]:
X.shape

(183428, 28)

In [ ]:
Y.shape

(183428, 42)

In [ ]:
Y.shape[1]

42

In [ ]:
seq_len = 6

In [ ]:
x = []
y = []

In [ ]:
for i in range(seq_len, len(X)):
    x.append(X[i-seq_len:i])
    
    yt = Y[i-seq_len:i]
    yr = []
    res = np.sum(yt, axis = 0)
    for i in range(0, len(res)):
        if(res[i] ==0):
            yr.append(0)
        else:
            yr.append(1)
    y.append(yr)

In [ ]:
type(x)

list

In [ ]:
x1 = np.asarray(x).astype('float32')

In [ ]:
x1.shape

(183422, 6, 28)

In [ ]:
y1 = np.asarray(y).astype('float32')

In [ ]:
y1.shape

(183422, 42)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x1, y1, train_size=0.8, random_state=42)


In [ ]:
X_train.shape

(146737, 6, 28)

In [ ]:
X_test.shape

(36685, 6, 28)

In [ ]:
y_train.shape

(146737, 42)

In [ ]:
y_test.shape

(36685, 42)

In [ ]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_test.shape[1]

In [ ]:
# model = Sequential()
# # Configuring the parameters
# model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
# model.add(LSTM(128, return_sequences=True))  
# # Adding a dropout layer
# model.add(Dropout(0.5))
# model.add(LSTM(64))
# model.add(Dropout(0.5))
# # Adding a dense output layer with sigmoid activation
# model.add(Dense(n_classes, activation='sigmoid'))
# model.summary()

In [ ]:
from tensorflow import keras
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import LSTM
import keras.models
from keras.layers import Activation, Dense
from keras.models import Sequential

In [ ]:
epochs = 20
batch_size =64

In [ ]:
model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               51600     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 42)                4242      
Total params: 65,942
Trainable params: 65,942
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# fit network
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
          verbose=0, validation_data=(X_test, y_test))


In [ ]:
# evaluate model
_, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)

In [ ]:
accuracy 

0.1975739449262619

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
Y_pred[0]

array([3.80333232e-09, 1.30074196e-09, 1.14741008e-04, 3.80000707e-15,
       1.09787345e-01, 4.30326909e-01, 1.17063450e-04, 9.98675227e-01,
       9.98646975e-01, 9.98659790e-01, 9.98666167e-01, 9.98647094e-01,
       9.98654962e-01, 9.98652935e-01, 9.98637557e-01, 9.98646617e-01,
       9.98671412e-01, 9.98641908e-01, 9.98670578e-01, 9.98664439e-01,
       9.98645544e-01, 9.98654246e-01, 9.98661637e-01, 9.98654008e-01,
       9.98653352e-01, 9.98658359e-01, 9.98644948e-01, 9.98639941e-01,
       9.98652339e-01, 9.98646021e-01, 9.98661757e-01, 9.98650670e-01,
       9.98646557e-01, 9.98636127e-01, 9.98652101e-01, 9.98641372e-01,
       9.98641133e-01, 9.98648167e-01, 9.98649120e-01, 9.98658836e-01,
       9.98666823e-01, 9.98638511e-01], dtype=float32)

In [ ]:
Y_pred[1]

array([8.3982300e-08, 7.9612654e-09, 9.6385556e-05, 1.0694541e-11,
       2.2889060e-01, 4.3515909e-01, 9.7792079e-05, 8.8229501e-01,
       8.8035321e-01, 8.8011765e-01, 8.7946892e-01, 8.7959361e-01,
       8.8061661e-01, 8.8021350e-01, 8.7989032e-01, 8.7998331e-01,
       8.8109767e-01, 8.8134301e-01, 8.8050443e-01, 8.8108301e-01,
       8.8021690e-01, 8.8063568e-01, 8.8086927e-01, 8.8142818e-01,
       8.7997639e-01, 8.8083386e-01, 8.7926584e-01, 8.7930238e-01,
       8.7914944e-01, 8.7918669e-01, 8.8125372e-01, 8.8064098e-01,
       8.8064694e-01, 8.7967277e-01, 8.8085330e-01, 8.7914717e-01,
       8.7992108e-01, 8.8064146e-01, 8.8051575e-01, 8.8030279e-01,
       8.8179302e-01, 8.8109779e-01], dtype=float32)